In [14]:
import pandas as pd
from enum import Enum
from dataclasses import dataclass
from math import floor
from datetime import time, datetime, timedelta
from time import sleep
from typing import Dict

In [2]:
SYMBOLS = ["SPY"]
PER_SYM_POSITION_LIM = 100
TICK_LENGTH = timedelta(seconds=1)

In [10]:
@dataclass
class BidAsk:
    bid: float
    ask: float
    
@dataclass 
class Order:
    class Side(Enum):
        Buy = 1
        Sell = 2
    class Type(Enum):
        Market = 1
        Limit = 2
    symbol: str
    side: Side
    quantity: int
    type: Type = Type.Market
    

In [4]:
class Strategy:
    def __init__(self, config, historical_data: pd.DataFrame, gateway):
        self.gateway = gateway

    # every tick, the Strategy receives as stimulus a new wave of prices for
    # each symbol (and hopefully makes us some $$$)
    def update(self, new_prices: pd.Series):
        pass
        
    @staticmethod
    def make(config, historical_data: pd.DataFrame, gateway):
        return Strategy(config, historical_data, gateway)

In [15]:
class MarketData:
    def __init__(self, login):
        pass
class OrderGateway:
    positions_: pd.Series
    floating_orders_: Dict[str, Order]
    
    def __init__(self, login):
        self.positions_ = pd.Series([0 for _ in SYMBOLS], index=SYMBOLS)
        
    def place_order(order: Order):
        raise "Not yet implemented"
    def liquidate():
        raise "Not yet implemented"
    def positions():
        return positions_.copy(deep=True)


In [6]:
def sleep_until(until: datetime):
    # division of one timedelta by another gives their ratio (in an int)
    ms_to_sleep_for = (until - datetime.now()) / timedelta(milliseconds=1)
    
    ms_per_second = 1000
    
    sleep(ms_to_sleep_for / ms_per_second)
    
def event_loop():
    # spin up MarketData, Gateway, and Strategy
    while True:
        tick_begin = datetime.now()
        tick_end = tick_begin + TICK_LENGTH
        print(f'start of tick: {tick_begin}')

        sleep_until(tick_end)


In [7]:
class BacktestingOrderGateway:
    positions_: pd.Series
    simulation_data_: pd.DataFrame
    tick_num_: int

    def __init__(self, simulation_data: pd.DataFrame):
        self.tick_num_ = 0
        self.simulation_data_ = simulation_data
        self.positions_ = pd.Series([0 for _ in SYMBOLS], index=SYMBOLS)
        self.pnl_ = 0

    # advance one tick (so that the current prices are the next)
    def advance(self):
        assert self.tick_num_ < self.simulation_data_.shape[0], "advanced past historical data count"
        self.tick_num_ += 1

    def place_order(self, order: Order) -> None:
        quotes = self.simulation_data_.iloc[self.tick_num_].loc
        assert order.symbol in SYMBOLS, f'unknown symbol: {order.symbol}'
        if order.side == Order.Side.Buy:
            exec_qty = min(order.quantity, PER_SYM_POSITION_LIM - self.positions_.loc[order.symbol])
            if(exec_qty != order.quantity):
                print(f"Reached position limit on {order=}")
            self.pnl_ -= exec_qty * quotes[order.symbol].ask
            self.positions_.loc[order.symbol] += exec_qty
        elif order.side == Order.Side.Sell:
            assert self.positions_.loc[order.symbol] >= order.quantity
            self.pnl_ += order.quantity * quotes[order.symbol].bid
            self.positions_[order.symbol] -= order.quantity
        #print(f'received order: {order=}. Positions after order: {self.positions_=}, Pnl after order: {self.pnl_}')

    def pnl(self)-> float:
        """
        Return the PNL up to the current tick.

        This assumes that we liquidate all of our current position
        """
        quotes = self.simulation_data_.iloc[self.tick_num_].loc
        return self.pnl_ + sum(quotes[sym].bid * self.positions_.loc[sym] for sym in SYMBOLS)

def backtest_quoter(config, historical_data: pd.DataFrame, make_stgy, training_pct = 0.5) -> pd.Series:
    """
    make_stgy is a function f: (Config, historical_data, gateway) -> Strategy

    Returns our PNL after each tick (of the backtest). Please don't just get the 
    tail of this list.
    """
    total_ticks = historical_data.shape[0]

    training_ticks = floor(training_pct * total_ticks)
    testing_ticks = total_ticks - training_ticks

    training_data = historical_data.iloc[0:training_ticks]
    testing_data = historical_data.iloc[training_ticks:]

    gateway = BacktestingOrderGateway(testing_data)
    stgy = make_stgy(config, training_data, gateway)

    pnl_table = pd.Series([-1 for _ in range(testing_ticks)], index=range(testing_ticks))
    
    for tick_num in range(testing_ticks):
        stgy.update(testing_data.iloc[tick_num])
        pnl_table.iloc[tick_num] = (gateway.pnl())
        gateway.advance()

    return pnl_table

In [16]:
def test_backtester():
    """
    Make sure our backtesting framework does what we think it does!
    """
    
    def close_to(x,y):
        return abs(x - y) < 0.0001
    """
    Test strategy that buys on one day, and then sells on the next.
    """
    class SimpleStrategy:
        def __init__(self, gateway):
            self.gateway_ = gateway
            self.tick_count_ = 0

        def update(self, _) -> None:
            side = Order.Side.Buy if self.tick_count_ % 2 == 0 else Order.Side.Sell
            self.gateway_.place_order(Order(SYMBOLS[0], side, 1))
            self.tick_count_ += 1

        @staticmethod
        def make(config, historical_data: pd.DataFrame, gateway):
            return SimpleStrategy(gateway)

    test_ticks = 8
    
    # all the prices are the same. Assuming test_ticks / 2 is even
    # we should end with PNL 0
    same_price_data = pd.DataFrame({
        SYMBOLS[0]: [BidAsk(9,9) for _ in range(test_ticks)]
    })
    same_price_data_pnl = backtest_quoter(None, same_price_data, SimpleStrategy.make, training_pct=0).iloc[-1]
    assert close_to(same_price_data_pnl, 0)

    # on even days, SPY costs TICK
    # on odd days, SPY sells for 2 * TICK
    alternating_price_data = pd.DataFrame({
        SYMBOLS[0]: [BidAsk(0, tick) if tick % 2 == 0 else BidAsk(2 * tick, float('inf')) for tick in range(test_ticks)]
    })
    alternating_price_data_pnl = backtest_quoter(None, alternating_price_data, SimpleStrategy.make, training_pct=0)
    expected_pnl = 0
    for tick, pnl in enumerate(alternating_price_data_pnl):
        if tick % 2 == 0:
            expected_pnl -= tick
        else:
            expected_pnl += 2 * tick
        assert close_to(pnl, expected_pnl)
test_backtester()

In [9]:
if __name__ == "__main__":
    event_loop()

start of tick: 2025-02-21 20:32:51.050485
start of tick: 2025-02-21 20:32:52.054360
start of tick: 2025-02-21 20:32:53.057330
start of tick: 2025-02-21 20:32:54.062347
start of tick: 2025-02-21 20:32:55.066595
start of tick: 2025-02-21 20:32:56.068360
start of tick: 2025-02-21 20:32:57.072119
start of tick: 2025-02-21 20:32:58.074877
start of tick: 2025-02-21 20:32:59.076431
start of tick: 2025-02-21 20:33:00.079007
start of tick: 2025-02-21 20:33:01.082136
start of tick: 2025-02-21 20:33:02.086412
start of tick: 2025-02-21 20:33:03.087811
start of tick: 2025-02-21 20:33:04.096142
start of tick: 2025-02-21 20:33:05.099558
start of tick: 2025-02-21 20:33:06.101479
start of tick: 2025-02-21 20:33:07.104829
start of tick: 2025-02-21 20:33:08.109951
start of tick: 2025-02-21 20:33:09.114216
start of tick: 2025-02-21 20:33:10.119045
start of tick: 2025-02-21 20:33:11.120834
start of tick: 2025-02-21 20:33:12.125948
start of tick: 2025-02-21 20:33:13.130178
start of tick: 2025-02-21 20:33:14

KeyboardInterrupt: 